In [44]:
import numpy as np
import pandas  as pd
import matplotlib.pyplot as plt
from scipy import stats as s

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
data.shape

(569, 33)

In [5]:
data.drop(columns = ["id" , "Unnamed: 32"] ,inplace = True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [7]:
data.drop(columns = data.columns[2:],inplace = True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   diagnosis    569 non-null    object 
 1   radius_mean  569 non-null    float64
dtypes: float64(1), object(1)
memory usage: 9.0+ KB


In [9]:
data[data["diagnosis"] == "B"].shape

(357, 2)

In [10]:
data[data["diagnosis"] == "M"].shape

(212, 2)

In [11]:
training_data_Len = int(0.7 * data.shape[0])
training_data_Len


398

In [12]:
B_training_data = data[data["diagnosis"] == 'B'].iloc[0:training_data_Len//2]

M_training_data = data[data["diagnosis"] == "M"].iloc[0:training_data_Len//2]

In [13]:
B_training_data.shape

(199, 2)

In [14]:
training_data =  pd.concat([B_training_data,M_training_data] ,axis = 0)

In [15]:
training_data

,diagnosis,radius_mean
19,B,13.540
20,B,13.080
21,B,9.504
37,B,13.030
46,B,8.196
...,...,...
499,M,20.590
501,M,13.820
503,M,23.090
509,M,15.460


In [16]:
cv_data_len = int(0.2 * data.shape[0])

In [17]:
cv_data_len

113

In [18]:
B_remaining_data = data[data["diagnosis"] == "B"].iloc[training_data_Len//2 : ]
    

M_reamaining_data = data[data["diagnosis"] == "M"].iloc[training_data_Len//2 :]    

In [19]:
B_remaining_data.shape

(158, 2)

In [20]:
M_reamaining_data.shape

(13, 2)

In [21]:
remaining_data = pd.concat([B_remaining_data,M_reamaining_data])

In [22]:
remaining_data

,diagnosis,radius_mean
360,B,12.54
361,B,13.30
362,B,12.76
363,B,16.50
364,B,13.40
...,...,...
563,M,20.92
564,M,21.56
565,M,20.13
566,M,16.60


In [23]:
cv_data = remaining_data.iloc[0:cv_data_len]

testing_data = remaining_data.iloc[cv_data_len:]

In [24]:
testing_data.shape

(58, 2)

 so we need to evaluate the following prob : 


# P(Diagnosis = M | radiusMean = x) ={ P(radiusMean = x | Diagnosis = M)\dot P(Diaganosis = M)}


In [37]:
mu_hat_rm = training_data[training_data["diagnosis"] == 'M']["radius_mean"].mean()

sigma_hat_rm = training_data[training_data["diagnosis"] == 'M']["radius_mean"].std()

In [39]:
M_prior = training_data[training_data["diagnosis"] == "M"].shape[0] / training_data.shape[0]

B_prior = training_data[training_data["diagnosis"] == "B"].shape[0] / training_data.shape[0]

In [36]:
mu_hat_rb = training_data[training_data["diagnosis"] == 'B']["radius_mean"].mean()

sigma_hat_rb = training_data[training_data["diagnosis"] == 'B']["radius_mean"].std()

In [47]:
def cv_data_mock_test(cv_data):
    
    radius_input = np.array(cv_data["radius_mean"])
    
    posterior_M = s.norm.pdf(radius_input,mu_hat_rm,sigma_hat_rm)*M_prior
    
    posterior_B = s.norm.pdf(radius_input,mu_hat_rb,sigma_hat_rb)*B_prior
    
    boolen_Mask = posterior_M > posterior_B
    
    predicted_cat = pd.Series(boolen_Mask)
    
    predicted_cat.replace(to_replace=[False,True],value = ["B" , "M"] ,inplace = True)
    
    
    return np.array(predicted_cat)

In [48]:
cv_result = cv_data_mock_test(cv_data)
cv_result

array(['B', 'B', 'B', 'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M',
       'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'B', 'M',
       'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'M'], dtype=object)

In [49]:
actual_result = np.array(cv_data["diagnosis"])
actual_result

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'], dtype=object)

In [51]:
boolen_mask = (cv_result == actual_result)
boolen_mask

array([ True,  True,  True, False,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True, False,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False])

In [52]:
accuracy  = np.count_nonzero(boolen_mask)/boolen_mask.shape[0]
accuracy

0.8495575221238938

Now Testing on Testing data and check it`s accuracy

In [53]:
def mock_test(data):
    
    radius_input = np.array(data["radius_mean"])
    
    posterior_M = s.norm.pdf(radius_input,mu_hat_rm,sigma_hat_rm)*M_prior
    
    posterior_B = s.norm.pdf(radius_input,mu_hat_rb,sigma_hat_rb)*B_prior
    
    boolen_Mask = posterior_M > posterior_B
    
    predicted_cat = pd.Series(boolen_Mask)
    
    predicted_cat.replace(to_replace=[False,True],value = ["B" , "M"] ,inplace = True)
    
    
    return np.array(predicted_cat)

In [54]:
testing_result = mock_test(testing_data)

In [56]:
actual_testing_data = np.array(testing_data["diagnosis"])

In [55]:
testing_result

array(['B', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'B', 'B', 'M', 'M', 'M', 'M', 'M', 'M', 'B',
       'M', 'M', 'M', 'M', 'M', 'M'], dtype=object)

In [57]:
boolen_mask_testing = (testing_result == actual_testing_data )

In [59]:
testing_accuracy = np.count_nonzero(boolen_mask_testing)/testing_data.shape[0]

testing_accuracy

0.896551724137931